# Transforming

In this notebook, we will explore how to use Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

## Setup

In [1]:

import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

api_key  = os.getenv('GOOGLE_API_KEY')

In [2]:
import requests
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
api_key = os.environ['GOOGLE_API_KEY']

# Set the API endpoint and API key.
endpoint = "https://generativelanguage.googleapis.com/v1beta"

def get_completion(prompt):
   # Prepare the request body.
   request_body = {
       "contents": [
           {
               "parts": [
                   {
                       "text": prompt
                   }
               ]
           }
       ]
   }

   # Make the request.
   response = requests.post(
       f"{endpoint}/models/gemini-pro:generateContent?key={api_key}",
       json=request_body,
   )

   # Parse the response.
   response_json = response.json()

   generated_content = response_json["candidates"][0]['content']['parts'][0]['text']

   # Return the generated content.
   return generated_content

## Translation

ChatGPT is trained with sources in many languages. This gives the model the ability to do translation. Here are some examples of how to use this capability.

In [5]:
prompt = f"""
Translate the following English text to Arbic: \ 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)

مرحبا أرغب في طلب خلاط


In [6]:
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

French


In [9]:
prompt = f"""
Translate the following  text to French and Hindi
and English pirate: \
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)

**French:**
```
Je veux commander un ballon de basket.
```

**Hindi:**
```
मैं बास्केटबॉल का ऑर्डर देना चाहता हूं।
```

**English pirate:**
```
Avast there, matey! I be wantin' to order a basketball.
```


In [11]:
prompt = f"""
Translate the following text to english in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

**Formal:**

> Would you like to place an order for a pillow?

**Informal:**

> Do you want to order a pillow?


### Universal Translator
Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!

In [12]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [15]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Hindi: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message (French): La performance du système est plus lente que d'habitude.
 **English:** System performance is slower than normal.

**Hindi:**  सिस्टम का परफॉर्मेंस नॉर्मल से धीमा है। 

Original message (Spanish): Mi monitor tiene píxeles que no se iluminan.
 **English**:  My monitor has unlit pixels.
 **Hindi**: मेरे मॉनिटर के पिक्सल लाइट नहीं हो रहे हैं। 

Original message (Italian): Il mio mouse non funziona
**English:** 
 My mouse is not working

**Hindi:**
 मेरा माउस काम नहीं कर रहा 

Original message (Polish): Mój klawisz Ctrl jest zepsuty
**English:**
My Ctrl key is broken

**Hindi:**
मेरी Ctrl key टूट चुकी है 

Original message (Chinese): 我的屏幕在闪烁
My screen is flickering

मेरी स्क्रीन टिमटिमा रही है। 



## Try it yourself!
Try some translations on your own!

## Tone Transformation
Writing can vary based on the intended audience. ChatGPT can produce different tones.


In [17]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Waseem, today which movie we going to watch.'
"""
response = get_completion(prompt)
print(response)

Dear [Recipient Name],

I hope this email finds you well.

My name is Waseem, and I am writing to inquire about the film selection for today's movie night.

I would appreciate it if you could provide a list of the available movies and indicate their starting times.

Thank you for your prompt attention to this matter.

Sincerely,
Waseem


## Format Conversion
ChatGPT can translate between formats. The prompt should describe the input and output formats.

In [19]:
data_json = { "resturant employees" :[ 
    {"name":"BMW", "email":"shyamjaiswal@gmail.com"},
    {"name":"ASh", "email":"bob32@gmail.com"},
    {"name":"San", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

```html
<!DOCTYPE html>
<html>
<head>
  <title>Restaurant Employees</title>
  <style>
    table {
      width: 100%;
      border-collapse: collapse;
    }

    th, td {
      border: 1px solid black;
      padding: 5px;
    }
  </style>
</head>
<body>
  <h1>Restaurant Employees</h1>

  <table>
    <thead>
      <tr>
        <th>Name</th>
        <th>Email</th>
      </tr>
    </thead>
    <tbody>
      {% for employee in employees %}
      <tr>
        <td>{{ employee.name }}</td>
        <td>{{ employee.email }}</td>
      </tr>
      {% endfor %}
    </tbody>
  </table>
</body>
</html>
```


In [20]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
{{ employee.name }},{{ employee.email }}


## Spellcheck/Grammar check.

Here are some examples of common grammar and spelling problems and the LLM's response. 

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [24]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

The girl with the black and white puppies has a ball.
No errors found
Its going to be a long day Does the car need its oil changed
Their goes -> There goes
There going -> They're going
they're -> their

There goes my freedom. They're going to bring their suitcases.
You're going to need your notebook.
That medicine affects my ability to sleep. Have you heard of the butterfly effect?
No errors found


In [25]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

Got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it, though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


In [3]:
text = f"""
Meet Aryan, a middle-class boy who faces relentless struggles to maintain a decent standard of living without resorting to loans/
 Despite financial hardships, he refuses to compromise his integrity/ Aryan works tirelessly, taking on multiple part-time jobs while juggling his studies/ He embraces frugality, cutting down on expenses wherever possible/ Through perseverance and determination, Aryan navigates life's challenges with resilience/ He dreams of a better future, where his hard work will finally pay off/ Despite the odds stacked against him, Aryan remains steadfast in his pursuit of success, proving that determination and grit can overcome any obstacle.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

Meet Aryan, a middle-class boy who faces relentless struggles to maintain a decent standard of living without resorting to loans. Despite financial hardships, he refuses to compromise his integrity. Aryan works tirelessly, taking on multiple part-time jobs while juggling his studies. He embraces frugality, cutting down on expenses wherever possible. Through perseverance and determination, Aryan navigates life's challenges with resilience. He dreams of a better future, where his hard work will finally pay off. Despite the odds stacked against him, Aryan remains steadfast in his pursuit of success, proving that determination and grit can overcome any obstacle.


In [26]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

Got this for my daughter for her birthday <span style='color:red;font-weight:700;text-decoration:line-through;'>cuz </span><span style='color:green;font-weight:700;'>because </span>she keeps taking mine from my <span style='color:red;font-weight:700;text-decoration:line-through;'>room.  </span><span style='color:green;font-weight:700;'>room. </span>Yes, adults also like pandas <span style='color:red;font-weight:700;text-decoration:line-through;'>too.  </span><span style='color:green;font-weight:700;'>too. </span>She takes it everywhere with her, and it's super soft and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute.  </span><span style='color:green;font-weight:700;'>cute. </span>One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for <span style='color:red;font-weight:700;text-decoration:line-through;'>it </span><span style='color:green;font-weight:700;'>it, </span>though. I think there might be other options that are bigger for the same <span style='color:red;font-weight:700;text-decoration:line-through;'>price.  </span><span style='color:green;font-weight:700;'>price. </span>It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.

In [6]:
from redlines import Redlines
from IPython.display import Markdown

diff = Redlines(text, response)
display(Markdown(diff.output_markdown))


Meet Aryan, a middle-class boy who faces relentless struggles to maintain a decent standard of living without resorting to <span style='color:red;font-weight:700;text-decoration:line-through;'>loans/ ¶ </span><span style='color:green;font-weight:700;'>loans. </span>Despite financial hardships, he refuses to compromise his <span style='color:red;font-weight:700;text-decoration:line-through;'>integrity/ </span><span style='color:green;font-weight:700;'>integrity. </span>Aryan works tirelessly, taking on multiple part-time jobs while juggling his <span style='color:red;font-weight:700;text-decoration:line-through;'>studies/ </span><span style='color:green;font-weight:700;'>studies. </span>He embraces frugality, cutting down on expenses wherever <span style='color:red;font-weight:700;text-decoration:line-through;'>possible/ </span><span style='color:green;font-weight:700;'>possible. </span>Through perseverance and determination, Aryan navigates life's challenges with <span style='color:red;font-weight:700;text-decoration:line-through;'>resilience/ </span><span style='color:green;font-weight:700;'>resilience. </span>He dreams of a better future, where his hard work will finally pay <span style='color:red;font-weight:700;text-decoration:line-through;'>off/ </span><span style='color:green;font-weight:700;'>off. </span>Despite the odds stacked against him, Aryan remains steadfast in his pursuit of success, proving that determination and grit can overcome any obstacle.

In [27]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

**Proofread and Corrected Review**

**Compelling Review:**

This adorable panda plush has become an indispensable companion for my daughter, who has an unwavering fondness for these beloved animals. Its irresistibly soft texture and charming asymmetry, with one ear slightly lower than the other, add to its endearing appeal.

She treasures this plush, carrying it with her wherever she goes, a testament to its exceptional quality and durability. However, its modest size may be a consideration at the given price point, as there may be larger options available for a similar investment.

Despite this minor caveat, the early arrival of the plush allowed me to experience its snuggly charms firsthand before gifting it to my daughter. Its exceptional softness and attention to detail make it an exceptional choice for those seeking a comforting or decorative companion.

In [7]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

**Advanced Academic Review**

**Title:** The Indomitable Spirit of Aryan: A Tale of Resilience and Determination

**Introduction:**

Introducing Aryan, a young man from humble beginnings whose unwavering resolve to maintain his dignity and aspirations amidst relentless financial challenges captivates readers. His story is a testament to the extraordinary power of perseverance and the indomitable spirit that resides within us all.

**Financial Hardships:**

Despite facing constant economic adversity, Aryan refuses to succumb to despair. His unwavering belief in his own worth drives him to work tirelessly, juggling multiple part-time jobs while maintaining his academic pursuits. This arduous schedule showcases his remarkable fortitude and resilience in the face of adversity.

**Embracing Frugality:**

Recognizing the need to manage his limited resources wisely, Aryan judiciously prioritizes his expenses. He embraces frugality, meticulously scrutinizing each purchase to ensure its necessity. This disciplined approach underscores his unwavering determination to navigate life's challenges with dignity and financial prudence.

**Resilience and Determination:**

Aryan's unwavering belief in his own abilities and the power of hard work empowers him to overcome the formidable obstacles that confront him. He remains steadfast in his pursuit of a better future, knowing that his dedication and perseverance will ultimately yield positive outcomes.

**Dreams and Aspirations:**

Despite the odds stacked heavily against him, Aryan harbors an unyielding dream of a life free from financial hardship. His unwavering optimism serves as a beacon of hope, propelling him forward even in the most trying of times.

**Conclusion:**

Aryan's story is a poignant reminder that determination and grit can triumph over adversity. His indomitable spirit inspires all who encounter him, demonstrating that with unwavering resolve and a relentless belief in one's abilities, any obstacle can be overcome.

## Try it yourself!
Try changing the instructions to form your own review.

Thanks to the following sites:

https://writingprompts.com/bad-grammar-examples/


# summary

#### 1.Translation
- LLMs can translate text between languages, as demonstrated with examples translating English to Spanish and French, and identifying the language of a given text.

#### 2.Universal Translator
- A scenario simulates a universal translator for IT support, where the model identifies languages, translates issues, and provides responses in multiple languages.

#### 3.Tone Transformation
- LLMs can adjust the tone of text, demonstrated by transforming informal slang into a formal business letter.

#### 4.Format Conversion
- The model can convert data between formats, such as translating a Python dictionary into an HTML table.

#### 5.Spellcheck/Grammar Check
- This task asks the model to correct errors in a text given within triple backticks. If no errors are found, it should respond with "No errors found." Also, it instructs not to use punctuation around the text.

#### Example
- A review of a product is proofread, corrected, and made more compelling, adhering to APA style and targeting an advanced reader.

